# SHOPPING DATASET

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
from csv import writer

!pip install geopy
!pip install folium

import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

# run a python file 
%run ../src/cleaning.py

In [2]:
df = pd.read_csv('../data/interim/shopping.csv')
df = df.dropna()
df

,description,image,image_link,name,name_link
0,"First opened in 1892, this ornate arcade featu...",https://www.travelvictoria.com.au/images/town/...,https://www.theblock.com.au/,Block Arcade,https://www.theblock.com.au/
1,"Located at the ""Paris end"" of Collins Street, ...",https://www.travelvictoria.com.au/images/town/...,https://www.collinsplace.com.au/,Collins Place,https://www.collinsplace.com.au/
2,Leafy Collins Street extends almost 2 kilometr...,https://www.travelvictoria.com.au/images/town/...,https://www.collinsstreet.com.au/,Collins Street,https://www.collinsstreet.com.au/
3,"Directly connected to Melbourne Central, Myer ...",https://www.travelvictoria.com.au/images/town/...,https://www.emporiummelbourne.com.au/,Emporium Melbourne,https://www.emporiummelbourne.com.au/
4,This is the city centre's major retail hub. It...,https://www.travelvictoria.com.au/images/town/...,https://www.melbournecentral.com.au/,Melbourne Central,https://www.melbournecentral.com.au/
5,Melbourne's general post office building began...,https://www.travelvictoria.com.au/images/town/...,https://www.melbournesgpo.com/,Melbourne's GPO,https://www.melbournesgpo.com/
6,"Established in 1878, this open-air market has ...",https://www.travelvictoria.com.au/images/town/...,https://www.qvm.com.au/,Queen Victoria Market,https://www.qvm.com.au/
7,Occupying the former site of the Queen Victori...,https://www.travelvictoria.com.au/images/town/...,https://www.qv.com.au/,QV Melbourne,https://www.qv.com.au/
8,"Built in 1869, Royal Arcade is a Melbourne ico...",https://www.travelvictoria.com.au/images/town/...,https://www.royalarcade.com.au/,Royal Arcade,https://www.royalarcade.com.au/
9,"Located in Collins Street, this shopping compl...",https://www.travelvictoria.com.au/images/town/...,https://www.stcollinslane.com/,St Collins Lane,https://www.stcollinslane.com/


In [3]:
# Declare a list that is to be converted into a column
address = ['282 Collins St, Melbourne VIC 3000, Australia', '45 Collins St, Melbourne VIC 3000, Australia', 'Collins Street, Melbourne', 
           '287 Lonsdale St, Melbourne VIC 3000, Australia', 'Swanston St, Melbourne VIC 3000, Australia', '350 Bourke St, Melbourne VIC 3000, Australia',
          'Queen St, Melbourne VIC 3000, Australia', 'Cnr Lonsdale Street and, Swanston St, Melbourne VIC 3000, Australia', '335 Bourke St, Melbourne VIC 3000, Australia',
          '260 Collins Street Melbourne, VIC 3000']

# Using 'Address' as the column name
# and equating it to the list
df['address'] = address

# Converting addresses to geographic information — Latitude and Longitude

In [4]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent="scrape")

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['address'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df.head()

,description,image,image_link,name,name_link,address,location,point
0,"First opened in 1892, this ornate arcade featu...",https://www.travelvictoria.com.au/images/town/...,https://www.theblock.com.au/,Block Arcade,https://www.theblock.com.au/,"282 Collins St, Melbourne VIC 3000, Australia","(Haigh’s Chocolates, 282, Collins Street, Melb...","(-37.8157783, 144.9649652, 0.0)"
1,"Located at the ""Paris end"" of Collins Street, ...",https://www.travelvictoria.com.au/images/town/...,https://www.collinsplace.com.au/,Collins Place,https://www.collinsplace.com.au/,"45 Collins St, Melbourne VIC 3000, Australia","(Kino Cinemas, 45, Collins Street, Melbourne, ...","(-37.8142466, 144.972547, 0.0)"
2,Leafy Collins Street extends almost 2 kilometr...,https://www.travelvictoria.com.au/images/town/...,https://www.collinsstreet.com.au/,Collins Street,https://www.collinsstreet.com.au/,"Collins Street, Melbourne","(Collins Street, Melbourne, City of Melbourne,...","(-37.813658, 144.9729712, 0.0)"
3,"Directly connected to Melbourne Central, Myer ...",https://www.travelvictoria.com.au/images/town/...,https://www.emporiummelbourne.com.au/,Emporium Melbourne,https://www.emporiummelbourne.com.au/,"287 Lonsdale St, Melbourne VIC 3000, Australia","(Calia, Lonsdale Street, Melbourne, City of Me...","(-37.8121269, 144.9641072, 0.0)"
4,This is the city centre's major retail hub. It...,https://www.travelvictoria.com.au/images/town/...,https://www.melbournecentral.com.au/,Melbourne Central,https://www.melbournecentral.com.au/,"Swanston St, Melbourne VIC 3000, Australia","(Swanston Street, Melbourne Innovation Distric...","(-37.8090866, 144.9635351, 0.0)"


In [5]:
print(df.columns)
# split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.head()

Index(['description', 'image', 'image_link', 'name', 'name_link', 'address',
       'location', 'point'],
      dtype='object')


,description,image,image_link,name,name_link,address,location,point,latitude,longitude,altitude
0,"First opened in 1892, this ornate arcade featu...",https://www.travelvictoria.com.au/images/town/...,https://www.theblock.com.au/,Block Arcade,https://www.theblock.com.au/,"282 Collins St, Melbourne VIC 3000, Australia","(Haigh’s Chocolates, 282, Collins Street, Melb...","(-37.8157783, 144.9649652, 0.0)",-37.815778,144.964965,0.0
1,"Located at the ""Paris end"" of Collins Street, ...",https://www.travelvictoria.com.au/images/town/...,https://www.collinsplace.com.au/,Collins Place,https://www.collinsplace.com.au/,"45 Collins St, Melbourne VIC 3000, Australia","(Kino Cinemas, 45, Collins Street, Melbourne, ...","(-37.8142466, 144.972547, 0.0)",-37.814247,144.972547,0.0
2,Leafy Collins Street extends almost 2 kilometr...,https://www.travelvictoria.com.au/images/town/...,https://www.collinsstreet.com.au/,Collins Street,https://www.collinsstreet.com.au/,"Collins Street, Melbourne","(Collins Street, Melbourne, City of Melbourne,...","(-37.813658, 144.9729712, 0.0)",-37.813658,144.972971,0.0
3,"Directly connected to Melbourne Central, Myer ...",https://www.travelvictoria.com.au/images/town/...,https://www.emporiummelbourne.com.au/,Emporium Melbourne,https://www.emporiummelbourne.com.au/,"287 Lonsdale St, Melbourne VIC 3000, Australia","(Calia, Lonsdale Street, Melbourne, City of Me...","(-37.8121269, 144.9641072, 0.0)",-37.812127,144.964107,0.0
4,This is the city centre's major retail hub. It...,https://www.travelvictoria.com.au/images/town/...,https://www.melbournecentral.com.au/,Melbourne Central,https://www.melbournecentral.com.au/,"Swanston St, Melbourne VIC 3000, Australia","(Swanston Street, Melbourne Innovation Distric...","(-37.8090866, 144.9635351, 0.0)",-37.809087,144.963535,0.0


In [6]:
# drop unecessary columns
print(f'List of existing columns: {df.columns.to_list()}\n\n')
df = df.drop(['location', 'point', 'altitude'], axis=1)
print(f'List of existing columns after dropping: {df.columns.to_list()}\n\n')

# filling a null values using fillna() 
df["latitude"].fillna("-37.81067565", inplace = True) 
# filling a null values using fillna() 
df["longitude"].fillna("144.965706939704", inplace = True) 

List of existing columns: ['description', 'image', 'image_link', 'name', 'name_link', 'address', 'location', 'point', 'latitude', 'longitude', 'altitude']


List of existing columns after dropping: ['description', 'image', 'image_link', 'name', 'name_link', 'address', 'latitude', 'longitude']




In [8]:
df.to_csv('../data/interim/shopping.csv', index=False)
df.head()

,description,image,image_link,name,name_link,address,latitude,longitude
0,"First opened in 1892, this ornate arcade featu...",https://www.travelvictoria.com.au/images/town/...,https://www.theblock.com.au/,Block Arcade,https://www.theblock.com.au/,"282 Collins St, Melbourne VIC 3000, Australia",-37.815778,144.964965
1,"Located at the ""Paris end"" of Collins Street, ...",https://www.travelvictoria.com.au/images/town/...,https://www.collinsplace.com.au/,Collins Place,https://www.collinsplace.com.au/,"45 Collins St, Melbourne VIC 3000, Australia",-37.814247,144.972547
2,Leafy Collins Street extends almost 2 kilometr...,https://www.travelvictoria.com.au/images/town/...,https://www.collinsstreet.com.au/,Collins Street,https://www.collinsstreet.com.au/,"Collins Street, Melbourne",-37.813658,144.972971
3,"Directly connected to Melbourne Central, Myer ...",https://www.travelvictoria.com.au/images/town/...,https://www.emporiummelbourne.com.au/,Emporium Melbourne,https://www.emporiummelbourne.com.au/,"287 Lonsdale St, Melbourne VIC 3000, Australia",-37.812127,144.964107
4,This is the city centre's major retail hub. It...,https://www.travelvictoria.com.au/images/town/...,https://www.melbournecentral.com.au/,Melbourne Central,https://www.melbournecentral.com.au/,"Swanston St, Melbourne VIC 3000, Australia",-37.809087,144.963535
